# AI Module

This notebook defines the AI-powered transcription editing system. It uses a Chat model to intelligently distinguish between new transcription content and edit commands.

In [ ]:
#| default_exp ai

In [ ]:
#| export
from lisette import *

True

## TranscriptEditor Class

The `TranscriptEditor` class manages live transcription with AI-assisted editing capabilities. It uses a Chat instance that maintains conversation history, allowing it to understand context across multiple chunks of transcription.

In [ ]:
#| export
class TranscriptEditor:
    """Manages live transcription with AI-assisted editing capabilities."""
    
    def __init__(self, model: str, temperature: float = 0.1):
        self.chat = Chat(
            model,
            sp="""You are helping with live transcription. As the user speaks, you'll receive each transcribed chunk (Each chunk being a line of text).
Your job is to:
1. Detect when the user wants to edit previous text (e.g., "change that to...", "delete the last part", "replace hamburgers with pizza")
2. When an edit is requested, return ONLY the complete corrected/edited transcript
3. When it's just new text, return ONLY the word "APPEND"
4. Keep the conversation history to understand context

Format your responses as:
- For edits: Return the full corrected transcript, each sentence on its own line
- For new text: APPEND""",
            temp=temperature
        )
        self.full_transcript = ""
        self.total_tokens = 0
    
    def process_chunk(self, chunk: str) -> dict:
        """Process a transcription chunk and determine if it's new text or an edit."""
        
        # Send the chunk with context about current transcript
        response = self.chat(chunk)
        
        result = response.choices[0].message.content.strip()
        tokens = response.usage.total_tokens if hasattr(response, "usage") else 0
        self.total_tokens += tokens
        
        # Determine if it's an append or edit
        if result.startswith("APPEND"):
            self.full_transcript += result
            action = "append"
        else:
            # It's an edit - replace full transcript
            self.full_transcript = result
            action = "edit"
        
        return {
            "transcript": self.full_transcript,
            "action": action,
            "tokens_used": tokens,
            "total_tokens": self.total_tokens
        }
    
    def get_transcript(self) -> str:
        """Get the current full transcript."""
        return self.full_transcript
    
    def reset(self):
        """Reset the transcript and chat history."""
        self.full_transcript = ""
        self.total_tokens = 0
        self.chat = Chat(
            self.chat.model,
            sp=self.chat.sp,
            temp=self.chat.temp
        )

### How it Works

The class initializes with a `Chat` instance that has a system prompt instructing it to:
1. **Detect edit commands** like "change that to...", "delete the last part", etc.
2. **Return "APPEND"** when the user is just adding new text
3. **Return the full corrected transcript** when an edit is requested

**Key Feature**: The `Chat` instance maintains a `hist` (history) of all previous interactions. This allows it to "remember" the full context of the conversation within the same Chat instance, enabling it to understand references like "that", "the last part", or "the first sentence" when processing edit commands.

The `process_chunk()` method:
- Sends each new transcription chunk to the AI
- Tracks token usage across all requests
- Determines if the response is an append or edit action
- Updates the full transcript accordingly

In [ ]:
#| eval: false
from dotenv import load_dotenv
load_dotenv()
# Test the TranscriptEditor
editor = TranscriptEditor("openai/gpt-4o-mini")

# Simulate transcription chunks
chunks = [
    "My name is Batman.\n",
    "I love pizza.\n",
    "This transcriber is working quite well.",
    "Actually, change pizza to hamburgers.",
    "Maybe even delete that first sentence about my name."
]

for chunk in chunks:
    result = editor.process_chunk(chunk)
    print(f"\n--- Chunk: {chunk}")
    print(f"Action: {result['action']}")
    print(f"Current transcript:\n{result['transcript']}")
    print(f"Tokens used: {result['tokens_used']}")


--- Chunk: My name is Batman.

Action: append
Current transcript:

Tokens used: 155

--- Chunk: I love pizza.

Action: append
Current transcript:

Tokens used: 169

--- Chunk: This transcriber is working quite well.
Action: append
Current transcript:

Tokens used: 187

--- Chunk: Actually, change pizza to hamburgers.
Action: edit
Current transcript:
My name is Batman.  
I love hamburgers.  
This transcriber is working quite well.
Tokens used: 223

--- Chunk: Maybe even delete that first sentence about my name.
Action: edit
Current transcript:
I love hamburgers.  
This transcriber is working quite well.
Tokens used: 255


## Testing the TranscriptEditor

This test demonstrates the editor's ability to:
1. **Append new transcription** - "My name is Batman", "I love pizza", etc.
2. **Handle edit commands** - "change pizza to hamburgers" modifies existing text
3. **Process complex edits** - "delete that first sentence" removes content

Notice how the AI understands contextual references because the Chat maintains conversation history (`hist`). When you say "change pizza to hamburgers", it knows what "pizza" refers to from earlier in the same Chat session.